# Goal
> Find divergences between nifty50 and option premiums so that I can utilize those divergences and market behaviors for my benefit.

I can may be understand the reasons why that's happening and find out a way to identify them before they happen and make profit by taking trade for the expectation.

## Divergence
When nifty moves up, calls are expected to move up based on their greeks and puts are supposed to go down. Are there times when that doesn't happen because of whatever reason. If yes, I would like to identify such situations and make profit.

## How to calculate divergence?
Inputs
1. Nifty 50 minute data
2. Option chain data
3. Option premiums minute data
4. Calculate for each minute candle for option, % change in nifty50, what's the % change in premium
5. Calculate DMA(x, y) for the % premium change
6. Deduct DMA for % change premium
8. Find candles which are outliers by getting 1 candle for each day and evaluate and see

In [72]:
import datetime as dt
import utils as ut
import constants as ct
import pandas as pd
import icharts as ic
from functools import cache
from pytz import timezone


TEST_START = dt.datetime.strptime("2023-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ct.INTERVAL_MIN1
EXCHANGE = ct.EXCHANGE_NSE
pickle_file_name = f"outlier_ocdf_2024_02_17.pkl"
MIN_VALUE = 10 ** 7 # 1 cr - 7 zeros, 100cr - 9 zeros, Minimum contract volume by value required

# pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 500)
# pd.set_option("display.width", 2000)

In [73]:
def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
test_dates = all_dates_shuffled.iloc[train_size:]
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)

@cache
def get_intraday_data(date):
    return ut.get_data(symbol=SYMBOL, date=date, interval=INTERVAL, exchange=EXCHANGE)

@cache
def get_symbol_candles(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data

@cache
def get_first_candle_close(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].close

@cache
def get_last_trading_day(date):
    return ut.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=ct.EXCHANGE_NSE)

train_dates["expiry"] = pd.NA
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_closest_expiry(SYMBOL, row.name), axis=1)
nifty_candles = [{"trade_date": idx.date(), "candles": get_symbol_candles(IC_SYMBOL, idx)} for idx, row in train_dates.iterrows()]
nifty_candles = pd.DataFrame(nifty_candles)
nifty_candles.set_index("trade_date", inplace=True)


result = []
for i, row in train_dates.iterrows():
    ocdf = ic.get_oc_df(IC_SYMBOL, row.expiry, row.name)
    nifty_open = nifty_candles.loc[row.name.date()].iloc[0].iloc[0].open
    ocdf = ocdf.loc[((nifty_open - 700) < ocdf.index) & ((nifty_open + 700) > ocdf.index)]
    ocdf.loc[:,"trade_date"] = row.name
    for col, val in row.items():
        ocdf[col] = val
    result.append(ocdf)
ocdf = pd.concat(result)
print(ocdf.shape)
get_intraday_data.cache_clear()
get_symbol_candles.cache_clear()
get_first_candle_close.cache_clear()
get_last_trading_day.cache_clear()

(3416, 58)


In [74]:
separated_cp = []
for i, row in ocdf.iterrows():
    ce = {}
    for col, val in row.items():
        if col.startswith("ce_"):
            ce[col[3:]] = val
    ce["option_type"] = ct.OPTION_TYPE_CALL
    ce["expiry"] = row.expiry
    ce["oc_date"] = row.oc_date
    ce["strike_price"] = row.name
    separated_cp.append(ce)
    pe = {}
    for col, val in row.items():
        if col.startswith("pe_"):
            pe[col[3:]] = val
    pe["option_type"] = ct.OPTION_TYPE_PUT
    pe["expiry"] = row.expiry
    pe["oc_date"] = row.oc_date
    pe["strike_price"] = row.name
    separated_cp.append(pe)

ocdf = pd.DataFrame(separated_cp)
ocdf.set_index(["expiry", "strike_price", "option_type"], inplace=True)
print(ocdf.shape)

(6832, 28)


In [75]:
found = not_found = 0

# @ut.ct
def get_premium_df_expiry(expiry, strike_price, option_type):
    def remove_seconds(candle):
        return candle.replace(second=0)

    global found, not_found
    try:
        pr = ic.get_opt_pre_df(symbol=SYMBOL, expiry=expiry, cur_dt=None, strike_price=strike_price, option_type=option_type)
        pr = pr.loc[(pr.volume * pr.open > MIN_VALUE)]
        pr.index = pr.index.to_series().apply(remove_seconds)
        found += 1
    except FileNotFoundError:
        not_found +=1
        return pd.NA
    return pr

def get_premium_df(expiry, trade_date, strike_price, option_type):
    df = get_premium_df_expiry(expiry, strike_price, option_type)
    if type(df) != type(pd.NA):
        return df.loc[df.index.date == trade_date.date()]
    return df

ocdf.loc[:, "premium"] = pd.NA
ocdf.loc[:, "premium"] = ocdf.apply(lambda oc: get_premium_df(expiry=oc.name[0], trade_date=oc.oc_date, strike_price=oc.name[1], option_type=oc.name[2]), axis=1)
# ocdf.loc[:, "premium"] = ocdf.apply(lambda oc: get_premium_df(expiry=oc.name[0], trade_date=oc.oc_date, strike_price=oc.name[1], option_type=oc.name[2]), axis=1)
print(f"found: {found}, notf: {not_found}")
# get_premium_df_expiry.cache_clear()

found: 6827, notf: 5


In [76]:
ocdf = ocdf.loc[ocdf.premium.notna()]
ocdf = ocdf.loc[ocdf.premium.apply(lambda r: r.shape[0] != 0)]
print(ocdf.shape)

(3125, 29)


In [77]:
def set_pc(row):
    # if type(row) == type(pd.NA) or row.shape[0] == 0:
    #     return
    first_low = row.iloc[0].low
    row.loc[:,"pc_low"] = (row["low"] - first_low) / first_low
    row.loc[:,"pc_high"] = (row["high"] - first_low) / first_low

nifty_candles.candles.apply(set_pc)

trade_date
2023-01-02    None
2023-01-04    None
2023-01-09    None
2023-01-10    None
2023-01-13    None
2023-01-16    None
2023-01-18    None
2023-01-23    None
2023-01-24    None
2023-01-27    None
2023-01-30    None
2023-02-06    None
2023-02-07    None
2023-02-08    None
2023-02-10    None
2023-02-13    None
2023-02-14    None
2023-02-15    None
2023-02-17    None
2023-02-21    None
2023-02-24    None
2023-03-01    None
2023-03-02    None
2023-03-08    None
2023-03-09    None
2023-03-16    None
2023-03-22    None
2023-03-23    None
2023-03-29    None
2023-04-10    None
2023-04-11    None
2023-04-12    None
2023-04-13    None
2023-04-17    None
2023-04-21    None
2023-04-25    None
2023-04-26    None
2023-04-27    None
2023-04-28    None
2023-05-02    None
2023-05-05    None
2023-05-09    None
2023-05-10    None
2023-05-11    None
2023-05-17    None
2023-05-22    None
2023-05-24    None
2023-05-25    None
2023-05-26    None
2023-05-31    None
2023-06-01    None
2023-06-06    None
2

In [12]:
ocdf["premium"].apply(set_pc)

expiry      strike_price  option_type
2023-01-05  17800         C              None
            17900         C              None
                          P              None
            17950         P              None
            18000         C              None
                                         ... 
2023-12-28  21850         C              None
                          P              None
            21900         C              None
            21950         C              None
            22000         P              None
Name: premium, Length: 3125, dtype: object

In [78]:
def set_pt_diff_nifty(row):
    # row["pc_diff"] = row["pc_high"] - row["pc_low"]
    # row[row.open < row.close, "pc_diff"] = - row["pc_diff"]
    row["pt_diff"] = row["high"] - row["low"]
    row.loc[row.open > row.close, "pt_diff"] = - row["pt_diff"]

def set_pt_diff_oc(premium, is_call):
    # row["pc_diff"] = row["pc_high"] - row["pc_low"]
    # row[row.open < row.close, "pc_diff"] = - row["pc_diff"]
    premium["pt_diff"] = premium["high"] - premium["low"]
    premium.loc[((premium.open > premium.close) & is_call) | ((premium.open < premium.close) & (not is_call)), "pt_diff"] = - premium["pt_diff"]

ocdf.apply(lambda r: set_pt_diff_oc(r.premium, r.name[2] == ct.OPTION_TYPE_CALL), axis=1)
# ocdf.loc[ocdf.index.get_level_values("option_type") == ct.OPTION_TYPE_PUT, "premium"].loc["pt_diff"] = - ocdf.loc[ocdf.index.get_level_values("option_type") == ct.OPTION_TYPE_PUT, "premium"].loc["pt_diff"]
nifty_candles.apply(lambda row: set_pt_diff_nifty(row.candles), axis=1)

trade_date
2023-01-02    None
2023-01-04    None
2023-01-09    None
2023-01-10    None
2023-01-13    None
2023-01-16    None
2023-01-18    None
2023-01-23    None
2023-01-24    None
2023-01-27    None
2023-01-30    None
2023-02-06    None
2023-02-07    None
2023-02-08    None
2023-02-10    None
2023-02-13    None
2023-02-14    None
2023-02-15    None
2023-02-17    None
2023-02-21    None
2023-02-24    None
2023-03-01    None
2023-03-02    None
2023-03-08    None
2023-03-09    None
2023-03-16    None
2023-03-22    None
2023-03-23    None
2023-03-29    None
2023-04-10    None
2023-04-11    None
2023-04-12    None
2023-04-13    None
2023-04-17    None
2023-04-21    None
2023-04-25    None
2023-04-26    None
2023-04-27    None
2023-04-28    None
2023-05-02    None
2023-05-05    None
2023-05-09    None
2023-05-10    None
2023-05-11    None
2023-05-17    None
2023-05-22    None
2023-05-24    None
2023-05-25    None
2023-05-26    None
2023-05-31    None
2023-06-01    None
2023-06-06    None
2

In [79]:
def set_nifty_pc_diff(premium_df):
    def nifty_pc_diff(r):
        x = nifty_candles.loc[r.name.date(), "candles"]
        # x = nifty_candles.loc[nifty_candles.index.date == r.name.date(), "candles"].iloc[0]
        try:
            x = x.loc[r.name, "pc_diff"]
        except KeyError:
            return pd.NA
        return x

    def nifty_pt_diff(r):
        x = nifty_candles.loc[r.name.date(), "candles"]
        # x = nifty_candles.loc[nifty_candles.index.date == r.name.date(), "candles"].iloc[0]
        try:
            x = x.loc[r.name, "pt_diff"]
        except KeyError:
            return pd.NA
        return x

    # premium_df["nifty_pc_diff"] = premium_df.apply(nifty_pc_diff, axis=1)
    premium_df["nifty_pt_diff"] = premium_df.apply(nifty_pt_diff, axis=1)

# Define the IST timezone using pytz
# Set the timezone for the 'date_col' using tz_localize
ist_timezone = timezone('Asia/Kolkata')
def set_tz(row):
    # print(row.premium.iloc[0])
    # print(dir(row.premium.index))
    # row.premium.index = row.premium.index.tz_localize(ist_timezone)
    try:
        row.premium.index = row.premium.index.tz_localize(ist_timezone)
    except TypeError:
        pass

ocdf.apply(set_tz, axis=1)
# ocdf.apply(lambda r: clean_row_wrap(r.premium), axis=1)
ocdf.apply(lambda r: set_nifty_pc_diff(r.premium), axis=1)
print(ocdf.shape)

# ocdf.iloc[:5]["premium"].apply(set_nifty_pc_diff)
# ocdf.iloc[:5]["premium"]
# ocdf.iloc[0]["premium"]
# ocdf["premium"].iloc[0].iloc[0]

(3125, 29)


In [80]:
def set_pc_comparison(premium_df):
    premium_df["ac_delta"] = premium_df["pt_diff"] / premium_df["nifty_pt_diff"]

ocdf.apply(lambda r: set_pc_comparison(r.premium), axis=1)

expiry      strike_price  option_type
2023-01-05  17800         C              None
            17900         C              None
                          P              None
            17950         P              None
            18000         C              None
                                         ... 
2023-12-28  21850         C              None
                          P              None
            21900         C              None
            21950         C              None
            22000         P              None
Length: 3125, dtype: object

In [81]:
ocdf.to_pickle(pickle_file_name)

In [82]:
ocdf = pd.read_pickle(pickle_file_name)

In [71]:
ocdf["premium"].iloc[1]

,date_time,open,high,low,close,volume,unknown1,unknown2,unknown3,unknown4,pt_diff,nifty_pt_diff,ac_delta
date_time,,,,,,,,,,,,,
2023-01-02 09:30:00+05:30,02.01.23 09:30:59,238.40,247.35,236.55,237.50,43350,194900,255.08,237.90,238.45,-10.80,-24.0,0.45
2023-01-02 09:34:00+05:30,02.01.23 09:34:59,252.40,263.05,246.45,256.70,41250,208300,252.90,255.85,256.50,16.60,<NA>,<NA>
2023-01-02 09:35:00+05:30,02.01.23 09:35:59,252.70,260.10,251.40,257.75,46350,210650,253.13,258.45,259.00,8.70,<NA>,<NA>
2023-01-02 09:40:00+05:30,02.01.23 09:40:59,281.25,283.30,271.95,273.50,36650,206450,256.94,273.25,273.60,-11.35,<NA>,<NA>
2023-01-02 09:45:00+05:30,02.01.23 09:45:59,287.70,292.70,283.10,283.10,35950,244450,259.81,281.70,282.30,-9.60,25.55,-0.375734
2023-01-02 14:15:00+05:30,02.01.23 14:15:59,297.45,297.45,270.35,276.00,35300,246000,282.98,276.00,276.70,-27.10,-34.4,0.787791
2023-01-02 14:20:00+05:30,02.01.23 14:20:59,265.90,265.90,251.45,259.00,68600,262900,281.58,259.00,259.65,-14.45,<NA>,<NA>
2023-01-02 14:46:00+05:30,02.01.23 14:46:59,292.60,314.10,292.60,314.10,36150,263950,281.27,313.55,314.25,21.50,<NA>,<NA>


In [23]:
nifty_candles.iloc[0]

candles                                   open      high       low     close  volume
date                                                                     
2023-01-02 09:15:00+05:30  18131.70  18150.15  18117.55  18141.35       0
2023-01-02 09:16:00+05:30  18142.20  18145.70  18131.40  18135.65       0
2023-01-02 09:17:00+05:30  18133.25  18140.55  18120.65  18140.55       0
2023-01-02 09:18:00+05:30  18139.40  18139.40  18120.15  18130.75       0
2023-01-02 09:19:00+05:30  18130.30  18133.55  18113.05  18113.10       0
...                             ...       ...       ...       ...     ...
2023-01-02 15:25:00+05:30  18207.15  18210.90  18206.25  18208.90       0
2023-01-02 15:26:00+05:30  18208.35  18213.00  18208.05  18212.30       0
2023-01-02 15:27:00+05:30  18212.75  18214.90  18211.55  18213.40       0
2023-01-02 15:28:00+05:30  18213.75  18215.10  18208.65  18208.65       0
2023-01-02 15:29:00+05:30  18210.00  18211.40  18206.25  18207.10       0

[375 rows x 5 columns]
Nam

In [65]:
# # ocdf["premium"].iloc[0].iloc[0].name.date()
# # nifty_candles.loc[ocdf["premium"].iloc[0].iloc[0].name.date()]
# # nifty_candles.loc[ocdf["premium"].iloc[0].iloc[0].name.date()]
# pd.concat(nifty_candles.tolist())
# pd.concat([candle_df.set_index('trade_date') for trade_date, candle_df in nifty_candles], axis=0)


pandas.core.frame.DataFrame

In [36]:
ocdf.iloc[1].premium

,date_time,open,high,low,close,volume,unknown1,unknown2,unknown3,unknown4,pc_low,pc_high,pc_diff
date_time,,,,,,,,,,,,,
2023-01-02 09:15:59,02.01.23 09:15:59,3.50,3.95,2.50,3.30,63750,444800,3.27,3.25,3.30,0.00,0.58,0.58
2023-01-02 09:16:59,02.01.23 09:16:59,3.30,3.30,3.05,3.10,48550,444800,3.22,3.10,3.15,0.22,0.32,0.10
2023-01-02 09:17:59,02.01.23 09:17:59,3.15,3.35,3.10,3.20,34300,476350,3.22,3.20,3.25,0.24,0.34,0.10
2023-01-02 09:18:59,02.01.23 09:18:59,3.30,3.30,3.15,3.25,27850,476350,3.22,3.20,3.25,0.26,0.32,0.06
2023-01-02 09:19:59,02.01.23 09:19:59,3.20,3.40,3.20,3.35,64200,476350,3.24,3.35,3.40,0.28,0.36,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-02 15:25:59,02.01.23 15:25:59,2.05,2.10,1.95,1.95,47000,510750,2.65,1.95,2.00,-0.22,-0.16,0.06
2023-01-02 15:26:59,02.01.23 15:26:59,1.95,1.95,1.90,1.95,17000,510750,2.65,1.95,2.00,-0.24,-0.22,0.02
2023-01-02 15:27:59,02.01.23 15:27:59,1.95,2.00,1.95,2.00,13350,470150,2.65,1.95,2.00,-0.22,-0.20,0.02


In [4]:
# ocdf.iloc[1].premium.ac_delta

ocdf.iloc[1].premium.ac_delta.quantile(0.95)

147.01097714282645

In [84]:
outlier_df = []
# ocdf.iloc[1].premium.sort_values(by="ac_delta", ascending=False)

def append_outlier(premium_df, name, delta):
    global outlier_df
    qtl = premium_df.ac_delta.quantile(0.05)
    outs = premium_df.loc[(premium_df.ac_delta <= qtl)]
    outs = outs.loc[(premium_df.volume * premium_df.open > MIN_VALUE)]
    outs["oc_name"] = f"{name[0]} {name[1]} {name[2]}"
    outs["delta"] = delta
    outs["qtl_diff"] = outs.ac_delta - qtl
    outs["dtime"] = outs.index.values
    outlier_df += outs.to_dict("records")

ocdf.apply(lambda oc: append_outlier(oc.premium, oc.name, oc.delta), axis=1)

expiry      strike_price  option_type
2023-01-05  17800         C              None
            17900         C              None
                          P              None
            17950         P              None
            18000         C              None
                                         ... 
2023-12-28  21850         C              None
                          P              None
            21900         C              None
            21950         C              None
            22000         P              None
Length: 3125, dtype: object

In [85]:
outlier_df = pd.DataFrame(outlier_df)
outlier_df.set_index("dtime", inplace=True)
outlier_df.index = outlier_df.index.tz_localize("UTC").tz_convert(ist_timezone)

In [86]:
outlier_df.sort_values(by=['dtime', 'ac_delta'], inplace=True)

In [87]:
outlier_df.to_csv("outlier_df_final.csv")

In [58]:
pd.set_option("display.max_rows", 10000)
print(outlier_df.shape)
outlier_df.to_pickle("outlier_df_final.pkl")
outlier_df.head(500)

(23049, 16)


,date_time,open,high,low,close,volume,unknown1,unknown2,unknown3,unknown4,pt_diff,nifty_pt_diff,ac_delta,oc_name,delta,qtl_diff
dtime,,,,,,,,,,,,,,,,
2023-01-02 09:15:00+05:30,02.01.23 09:15:59,185.00,185.00,111.05,161.35,81900,420750,155.51,161.05,161.35,-73.95,32.60,-2.268405,2023-01-05 00:00:00 18050 C,0.7715,-1.258561
2023-01-02 09:15:00+05:30,02.01.23 09:15:59,129.00,153.00,111.20,134.85,1009200,4411550,140.99,134.40,134.70,-41.80,32.60,-1.282209,2023-01-05 00:00:00 18200 P,-0.4676,-0.691353
2023-01-02 09:15:00+05:30,02.01.23 09:15:59,134.05,144.55,105.55,131.10,815300,2222950,125.41,131.10,131.40,-39.00,32.60,-1.196319,2023-01-05 00:00:00 18100 C,0.6993,-0.380393
2023-01-02 09:15:00+05:30,02.01.23 09:15:59,277.00,302.90,274.15,278.70,37250,393000,288.98,277.45,278.25,-28.75,32.60,-0.881902,2023-01-05 00:00:00 18400 P,-0.7955,-0.862307
2023-01-02 09:15:00+05:30,02.01.23 09:15:59,62.00,88.70,60.95,66.30,304550,960250,69.99,66.40,66.55,-27.75,32.60,-0.851227,2023-01-05 00:00:00 18050 P,-0.2428,-0.513522
2023-01-02 09:15:00+05:30,02.01.23 09:15:59,160.00,185.80,160.00,165.95,137350,774250,171.40,165.25,165.90,-25.80,32.60,-0.791411,2023-01-05 00:00:00 18250 P,-0.5568,-0.091606
2023-01-02 09:15:00+05:30,02.01.23 09:15:59,196.90,221.55,196.90,200.05,223800,1080850,208.88,200.00,200.20,-24.65,32.60,-0.756135,2023-01-05 00:00:00 18300 P,-0.6407,-0.172104
2023-01-02 09:15:00+05:30,02.01.23 09:15:59,104.05,123.50,100.35,107.55,712900,1182000,112.89,107.55,107.75,-23.15,32.60,-0.710123,2023-01-05 00:00:00 18150 P,-0.3846,-0.193418
2023-01-02 09:15:00+05:30,02.01.23 09:15:59,103.00,109.95,87.95,102.95,852750,1152750,98.58,102.65,102.95,-22.00,32.60,-0.674847,2023-01-05 00:00:00 18150 C,0.6186,-0.001216
